In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from project_utils import *
from pygsp import graphs, filters

%load_ext autoreload
%autoreload 2

In [2]:
actors_agg_adj = np.load("sparse_agg_actor_adj.npy")
actors_agg_df = pd.read_pickle("actors_agg_df.pkl")
actors_graph = nx.from_numpy_matrix(actors_agg_adj)

In [3]:
G = graphs.Graph(actors_agg_adj)

In [4]:
#G.set_coordinates('spring')
#G.plot()

In [5]:
#Laplacian and spectral decomposition
G.compute_laplacian('combinatorial')
laplacian = G.L.toarray()
lam, U = np.linalg.eig(laplacian)

In [6]:
# Create ideal filters
ideal_hp = np.ones((actors_agg_adj.shape[0],))
ideal_hp[lam <= 0.1] = 0  # High-pass filter with cut-off at lambda=0.1

In [7]:
order = 3
coeff_hp = fit_polynomial(lam, order, ideal_hp)
g_hp = polynomial_graph_filter(coeff_hp, laplacian)

In [8]:
x_hp_polynomial = g_hp @ actors_agg_adj

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
# load communities obtained running notebook 05 
actors_dataset = np.load("actors_with_communities_dataset.npy")
y = actors_dataset[:,-1]
train_features,test_features,train_labels,test_labels=train_test_split(x_hp_polynomial,y,test_size=0.2)

In [11]:
# Fit a logistic regression model
max_iter = 10000000
std_scaler = StandardScaler()
scaled_train_features = std_scaler.fit_transform(train_features)
model = LogisticRegression(
    solver="liblinear", multi_class="auto", max_iter=max_iter, C=10000
)
model.fit(scaled_train_features, train_labels)
scaled_test_features = std_scaler.transform(test_features)
predict_train = model.predict(scaled_train_features)
predict_test = model.predict(scaled_test_features)

In [12]:
# Logistic regression accuracy on filtered adj matrix
accuracy = accuracy_score(test_labels,predict_test)
print("Accuracy: " +str(accuracy))

Accuracy: 0.9367088607594937
